In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [24]:
df=pd.read_csv('/content/balanced_adult_dataset.xls')
df.head(4)

,Age,Workclass,fnlwgt,education,marital-status,occupation,relationship,race,Gender,hours-per-week,native-country,Potential
0,31.0,' Private',157640.0,' HS-grad',' Married-civ-spouse',' Prof-specialty',' Husband',' White',' Male',55.0,' United-States',H
1,44.0,' Private',222504.0,' Some-college',' Married-civ-spouse',' Adm-clerical',' Wife',' White',' Female',45.0,' United-States',H
2,52.0,' Private',146567.0,' HS-grad',' Divorced',' Machine-op-inspct',' Not-in-family',' Black',' Male',40.0,' United-States',H
3,55.0,' Private',143266.0,' Assoc-voc',' Married-civ-spouse',' Craft-repair',' Husband',' White',' Male',50.0,' Hungary',H


In [19]:
#tranforming categorical data to numerical
from sklearn.preprocessing import LabelEncoder 
labelencoder= LabelEncoder() 
for column in df:
    df[column] = labelencoder.fit_transform(df[column]) 
df.head()

,Age,Workclass,fnlwgt,education,marital-status,occupation,relationship,race,Gender,hours-per-week,native-country,Potential
0,942,3,15182,11,2,9,0,4,1,16260,38,0
1,11841,3,26975,15,2,0,5,4,0,9297,38,0
2,18507,3,13565,11,0,6,1,2,1,2936,38,0
3,20053,3,13075,8,2,2,0,4,1,13826,17,0
4,441,3,31825,9,2,11,0,4,1,9297,38,0


In [20]:
#separating input and output attributes
X = df.iloc[:,:11].values
y = df.iloc[:,11:].values

In [21]:
#splitting into testing and training dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(len(X_train),len(X_test),len(y_train),len(y_test))

39528 9882 39528 9882


In [22]:
import tensorflow as tf

In [25]:
from keras.utils import to_categorical
num_classes = 2

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(y_train.shape)
print(y_train[0])
#print(len(y_train[0]))

(39528, 2)
[0. 1.]


In [27]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,SimpleRNN
from keras import optimizers
#from keras.preprocessing.sequence import pad_sequences
import numpy as np
#import numpy as np

#(samples, timesteps, features)
#(batch_size, timesteps, input_dim)
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)  #(750, 100, 1)

X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))
print(X_test.shape)

(39528, 11, 1)
(9882, 11, 1)


In [30]:
num_classes=2
maxlen=11
def vanilla_rnn():
    model = Sequential()
    #model.add(SimpleRNN(50, input_shape = (X_train.shape[1], X_train.shape[2],), return_sequences = False))
    model.add(SimpleRNN(50, input_shape = (maxlen,1), return_sequences = False))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    model.summary()
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [31]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn = vanilla_rnn, epochs = 5, batch_size = 50)
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  This is separate from the ipykernel package so we can avoid doing imports until


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 50)                2600      
                                                                 
 dense (Dense)               (None, 2)                 102       
                                                                 
 activation (Activation)     (None, 2)                 0         
                                                                 
Total params: 2,702
Trainable params: 2,702
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


791/791 [==============================] - 5s 4ms/step - loss: 0.6412 - accuracy: 0.5939
Epoch 2/5
791/791 [==============================] - 3s 4ms/step - loss: 0.6704 - accuracy: 0.5387
Epoch 3/5
791/791 [==============================] - 3s 4ms/step - loss: 0.6617 - accuracy: 0.5595
Epoch 4/5
791/791 [==============================] - 3s 4ms/step - loss: 0.6267 - accuracy: 0.6197
Epoch 5/5
791/791 [==============================] - 3s 4ms/step - loss: 0.5460 - accuracy: 0.7078


In [36]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

print(accuracy_score(y_pred, y_test_))

309/309 [==============================] - 1s 2ms/step
0.824428253390002


In [38]:
from sklearn.metrics import confusion_matrix, accuracy_score
print("Confusion Matrix:")
print(confusion_matrix(y_test_, y_pred))
print("Accuracy:",accuracy_score(y_test_, y_pred)*100,"%")

Confusion Matrix:
[[4887  117]
 [1618 3260]]
Accuracy: 82.44282533900021 %
